In [1]:
import pandas as pd
import numpy as np
import itertools
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from joblib import Parallel, delayed
from tqdm import tqdm
import time
import gc

In [2]:
def calcGeomAvg(returns: np.array,
    annualized: bool=False,
    periods_in_year: int=None) -> float: 
    """ Calculate the geometric average of a vector of simple returns.

    Args:
        returns (np.array): vector of a simple returns at any frequency.
        annualized (bool): whether to annualize the statistic.
        periods_in_year (int): how many periods of the given frequency are in a year.

    Returns:
        (float): scalar geometric average.
    """
    geom_avg_at_given_freq = np.prod(1+returns)**(1/len(returns))-1
    if annualized==False:
        return geom_avg_at_given_freq
    else:
        return (geom_avg_at_given_freq+1)**periods_in_year-1

def calcTSAvgReturn(returns: np.array,
    annualized: bool=False,
    periods_in_year: int=None) -> float:
    """ Calculate the time series mean return of a vector of simple returns with option to annualize.

    Args:
        returns (np.array): vector of a simple returns at any frequency.
        annualized (bool): whether to annualize the statistic.
        periods_in_year (int): how many periods of the given frequency are in a year.

    Returns:
        (float): scalar time series mean return.
    """
    mean_ret_at_given_freq = np.mean(returns)
    if annualized == False:
        return mean_ret_at_given_freq
    else:
        mean_ret = periods_in_year*mean_ret_at_given_freq
        if mean_ret < -1:
            return -1.
        else:
            return mean_ret

def calcTotalReturn(returns: np.array,
    annualized: bool=False,
    periods_in_year: int=None) -> float:
    """ Calculate the total return of a vector of simple returns with option to annualize.

    Args:
        returns (np.array): vector of a simple returns at any frequency.
        annualized (bool): whether to annualize the statistic.
        periods_in_year (int): how many periods of the given frequency are in a year.

    Returns:
        (float): scalar total return.
    """
    total_return = np.prod(1+returns)-1
    if annualized==False:
        return total_return
    else:
        return (total_return+1)**(periods_in_year/len(returns))-1

def calcSD(returns: np.array,
    annualized: bool=False,
    periods_in_year: int=None) -> float: 
    """ Calculate the standard deviation of a vector of simple returns with option to annualize.

    Args:
        returns (np.array): vector of a simple returns at any frequency.
        annualized (bool): whether to annualize the statistic.
        periods_in_year (int): how many periods of the given frequency are in a year.

    Returns:
        (float): scalar standard deviation.
    """
    sd_at_given_freq = np.std(returns)
    if annualized==False:
        return sd_at_given_freq
    else:
        return np.sqrt(periods_in_year)*sd_at_given_freq

def calcSharpe(returns: np.array,
    periods_in_year: int,
    risk_free_returns: np.array=None) -> float:
    """ Calculate the annual Sharpe Ratio of a vector of simple returns. 

    Args:
        returns (np.array): vector of a simple returns at any frequency.
        periods_in_year (int): how many periods of the given frequency are in a year.
        risk_free_returns (np.array): vector of simple returns of the risk free rate.

    Returns:
        (float): scalar standard deviation.
    """
    if risk_free_returns is not None:
        returns = returns - risk_free_returns
    
    return (calcTSAvgReturn(returns, annualized=True, periods_in_year=periods_in_year) /
            calcSD(returns, annualized=True, periods_in_year=periods_in_year))

In [3]:
def calcTransactionCosts(positions: np.array) -> np.array:
    ''' Calculate a vector of transaction costs which are positive numbers in return units.
    
    Args:
        positions (np.array): vector of positions, where positive is long and above 1, in absolute
                              value terms, is a leveraged position.

    Returns:
        tc (np.array): vector of transaction costs in return terms.
    '''
    # transaction costs, in return terms, from kraken for trading two spots paris, on margin
    tc_to_open        = 0.0015
    tc_to_close       = 0.0015
    tc_to_open_margin = 0.00015
    tc_margin_12_hr    = 0.0003

    # initial tc array
    tc = np.zeros(len(positions))

    # set first tc
    first_position = positions[0]
    if first_position == 0:
        tc[0] = 0
    elif (-1 <= first_position) & (first_position <= 1):
        tc[0] = tc_to_open
    elif (-5 <= first_position) & (first_position <= 5):
        tc[0] = tc_to_open+tc_to_open_margin+tc_margin_12_hr
    else:
        raise ValueError('first position is not a valid position.')

    # set remaining tc's
    for i in range(1,len(tc)):
        prev_position = positions[i-1]
        current_position = positions[i]
        if current_position == prev_position:
            if np.abs(current_position)>1:
                tc[i] = tc_margin_12_hr
        else:
            if current_position==0:
                tc[i] = tc_to_close
            elif (-1 <= current_position) & (current_position <= 1):
                tc[i] = tc_to_close+tc_to_open
            elif (-5 <= current_position) & (current_position <= 5):
                tc[i] = tc_to_close+tc_to_open+tc_to_open_margin+tc_margin_12_hr
            else: 
                raise ValueError('position '+str(i)+' is not a valid position.')

    # adjust last tc element for closing position
    last_position = positions[-1]
    if np.abs(last_position)>0:
        tc[-1] += tc_to_close

    return tc

In [4]:
def fitRF(train_df: pd.DataFrame,
    lhs_col: str,
    rhs_cols: list,
    hps: dict) -> RandomForestClassifier: 
    ''' Fit random forest on given training data using RHS to predict LHS with given hps.

    Args:
        train_df (pd.DataFrame): training data.
        lhs_col (str): LHS column to predict.
        rhs_cols (list): list of strings of RHS features.
        hps (dict): key value pairs for hyperparameters to set including:
                    `num_estimators`, `subsample`, `min_samp_leaf`, `max_depth`, 
                    `max_feature`, `loss_func`, and `weight_func`.

    Returns:
        (RandomForestClassifier): fitted model.
    '''
    # obtain the rhs and lhs data
    train_rhs   = train_df[rhs_cols].values.astype('float32')
    train_lhs   = train_df[lhs_col].values.reshape(-1).astype('int')
    
    # build the model
    model = RandomForestClassifier(n_estimators=hps['num_estimator'],
        criterion=hps['loss_func'],
        max_depth=hps['max_depth'],
        min_samples_leaf=hps['min_samp_leaf'],
        max_features=hps['max_feature'],
        n_jobs=4,
        random_state=int(hps['num_estimator']
            *hps['subsample']
            *hps['min_samp_leaf']
            *hps['max_depth']
            *hps['max_feature']),
        class_weight=hps['class_balance'],
        max_samples=hps['subsample'])

    # calculate sample weights as linearly spaced from 0 to max value s.t. it sums to one
    num_samples = train_lhs.shape[0]
    if hps['weight_func'] == 'linear':
        weights = np.arange(0, num_samples)
        weights = weights/np.sum(weights)
        epsilon = weights[1]/2
        weights[0] = epsilon
        weights[-1] -= epsilon
    elif hps['weight_func'] == 'uniform':
        weights = np.ones(num_samples)/num_samples
    elif hps['weight_func'] == 'log':
        weights = np.log(np.arange(1,num_samples+1))
        weights = weights / np.sum(weights)
    elif hps['weight_func'] == 'exp':
        weights = np.arange(num_samples)**1.1
        weights = weights / np.sum(weights)
    else:
        raise ValueError('the weights function must be set to linear, uniform, log, or exp.')

    # fit
    model.fit(X=train_rhs, 
        y=train_lhs, 
        sample_weight=weights)

    # obtain training yhats and accuracy
    train_yhats = model.predict(train_rhs)
    train_acc = model.score(train_rhs, train_lhs)

    return model, train_acc, train_yhats

def genRFYhats(oos_df: pd.DataFrame, 
    model: RandomForestClassifier, 
    lhs_col: str, 
    rhs_cols: list) -> int: 
    """ Generate predicted probabilities for input data using a random forest classifier model.

    Args:
        oos_df (pd.DataFrame): out of sample data to fit on.
        model (RandomForestClassifier): trained random forest model.
        lhs_col (str): name of the column containing the label data.
        rhs_cols (list): column names containing the feature data.

    Returns:
        (int): predicted scalar class.
    """
    # obtain the RHS data
    oos_rhs   = oos_df[rhs_cols].values.astype('float32')
    
    # form the yhats
    yhat = model.predict(oos_rhs)
    
    # Return results
    return yhat[0]

In [14]:
def runCV(df: pd.DataFrame, 
    lhs_col: str,
    val_start_date: str,
    num_cpus: int,
    arch_name: str, 
    out_csv_fp: str) -> list:
    ''' run step-forward cross validation to select optimal hyperparameters for target model 
        returning fitting yhats and models as well as outputting results to csv.

    Args:
        df (pd.DataFrame): panel of training and val data with date index, LHS variable named 
                           `lhs_col`, and remaining cols are RHS features.
        lhs_col (str): name of the lhs target column.
        val_start_date (str): the first date for the validation period.
        val_end_date (str): the last date for the validation period.
        num_cpus (int): number of cpus to use when parallelizing.
        arch_name (str): name of architecture to use when saving intermitent results.
        out_csv_fp (str): filepath to output the csv file without `.csv' on end.
          
    Returns:
        (list): list of dictionaries for each hyperparameter fit where each list contains keys of:
                    `hps` is dict of hyperparameter combination,
                    `yhats` is an array of validation period yhats, and,
                    `models` is list of fitted models.  
    '''
    # initialize args
    val_dates = np.unique(df[val_start_date:].index.values)
    results_list = []
    csv_dict_list = []
    rhs_cols = list(df.columns.values)
    rhs_cols.remove(lhs_col)

    # initialize hp grid for gbdt
    # TODO: dial in return threshold to 3 options [0, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.011, 0.012]
    # TODO: dial in max depth to 3 options
    # TODO: dial in num est to 3 options
    # TODO: dial in max feats to 3 options [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]
    # TODO: tune across num est, max depth, max feats, return threshold, and whether to balance classes
    # TODO: dial in subsamples [0.9, 0.95, 0.99, 0.999]
    # TODO dial in min sample leafs [5, 10, 20, 50, 100, 250, 500, 1000]
    # TODO: dial in 3 options for all: num est, max depth, max feat, loss funcs, weight funcs, class balance, and ret threshold
    #       loss funcs ['log_loss', 'gini', 'entropy'] and weight funs ['uniform', 'linear', 'log', 'exp'] 
    # TODO: combine all the results into csv 
    num_estimators  = [250]
    subsamples      = [0.95]
    min_samp_leafs  = [50]
    max_depths      = [2, 3, 4, 5, 6, 7]
    max_features    = [0.2]
    loss_funcs      = ['log_loss']
    weight_funcs    = ['linear']
    class_balances  = ['balanced'] 
    ret_thresholds  = list(np.arange(1,21)/1000)

    # loop over all hp combos
    for hps in itertools.product(num_estimators,
                                 subsamples,
                                 min_samp_leafs,
                                 max_depths, 
                                 max_features,
                                 loss_funcs,
                                 weight_funcs,
                                 class_balances,
                                 ret_thresholds):
        # initialize args
        results_dict = {}
        results_dict['hps'] = {'num_estimator': hps[0],
            'subsample': hps[1],
            'min_samp_leaf': hps[2],
            'max_depth': hps[3],
            'max_feature': hps[4],
            'loss_func': hps[5],
            'weight_func': hps[6],
            'class_balance': hps[7],
            'ret_threshold': hps[8]}
        print(results_dict['hps'], '\n') # monitor progress

        # form lhs
        cv_df = df.copy()
        ret_threshold = results_dict['hps']['ret_threshold']
        cv_df['y'] = 1
        cv_df.loc[cv_df[lhs_col] > ret_threshold, 'y'] = 2
        cv_df.loc[cv_df[lhs_col] < -ret_threshold, 'y'] = 0

        # remove actual returns and save val period returns
        btc_eth_diff = cv_df[val_start_date:][lhs_col].values
        cv_df = cv_df.drop(lhs_col, axis=1)

        # fit model on all val dates
        tic = time.perf_counter()
        def loopOverValDates(val_date): # set up as a func to loop over
            # form train and val data
            train_df = cv_df[cv_df.index < val_date].copy()
            val_df   = cv_df[cv_df.index == val_date].copy()

            # fit model and generate yhats for val week
            model, train_acc, train_yhats = fitRF(train_df, 'y', rhs_cols, hps=results_dict['hps'])
            yhats = genRFYhats(val_df, model, 'y', rhs_cols)

            return yhats, model, train_acc, train_yhats

        val_results = Parallel(n_jobs=int(num_cpus/4))(delayed(loopOverValDates)(val_date) for val_date in tqdm(val_dates))

        # extract validation periods results
        yhats_list = []
        models_list = []
        train_acc_list = []
        train_yhats_list = []
        for t in range(len(val_results)):
            yhats_list.append(val_results[t][0])
            models_list.append(val_results[t][1])
            train_acc_list.append(val_results[t][2])
            train_yhats_list.append(val_results[t][3])
        yhats = np.array(yhats_list)-1
        ys    = cv_df[val_start_date:].y.values-1
        results_dict['yhats'] = yhats
        results_dict['models'] = models_list

        # save results to master result list    
        results_list.append(results_dict)

        # save results to csv to monitor during cv
        toc = time.perf_counter()
        csv_dict = results_dict['hps'].copy()
        del results_dict
        csv_dict['arch_name'] = arch_name
        csv_dict['val_start_date'] = val_start_date
        csv_dict['val_end_date'] = np.datetime_as_string(np.max(df.index.values))[:10]
        csv_dict['runtime_mins'] = round((toc - tic)/60, 0) 
        total_obs = 0
        total_yhat_long = 0
        total_yhat_short = 0
        for train_yhats in train_yhats_list:
            total_obs += train_yhats.shape[0]
            total_yhat_long += np.sum(train_yhats==1)
            total_yhat_short += np.sum(train_yhats==-1)
        csv_dict['yhat_long_pct_train'] = total_yhat_long / total_obs
        csv_dict['yhat_short_pct_train'] = total_yhat_short / total_obs
        csv_dict['accuracy_train'] = np.mean(np.array(train_acc_list))
        csv_dict['yhat_long_pct'] = np.sum(yhats==1)/len(yhats)
        csv_dict['yhat_short_pct'] = np.sum(yhats==-1)/len(yhats)
        csv_dict['accuracy'] = np.sum(yhats == ys)/len(ys)
        tc = calcTransactionCosts(yhats)
        returns = yhats*btc_eth_diff - tc
        csv_dict['geom_mean'] = calcGeomAvg(returns)
        csv_dict['sharpe'] = calcSharpe(returns, periods_in_year=365*2)
        csv_dict['sd_ann'] = calcSD(returns, annualized=True, periods_in_year=365*2)
        csv_dict['ts_mean_ann'] = calcTSAvgReturn(returns, annualized=True, periods_in_year=365*2)
        for lev in [2, 3, 4, 5]:
            lev_yhats = yhats*lev
            lev_tc = calcTransactionCosts(lev_yhats)
            lev_returns = lev_yhats*btc_eth_diff - lev_tc 
            csv_dict['geom_mean_x'+str(lev)] = calcGeomAvg(lev_returns)
            csv_dict['sharpe_x'+str(lev)] = calcSharpe(lev_returns, periods_in_year=365*2)
            csv_dict['sd_ann_x'+str(lev)] = calcSD(lev_returns, annualized=True, periods_in_year=365*2)
            csv_dict['ts_mean_ann_x'+str(lev)] = calcTSAvgReturn(lev_returns, annualized=True, periods_in_year=365*2)
        csv_dict_list.append(csv_dict)
        results_df = pd.DataFrame(csv_dict_list)

        timestr = time.strftime("%Y%m%d_%H%M%S")
        fp = out_csv_fp+'_'+arch_name+'_'+timestr+'.csv'
        results_df.to_csv(fp, index=False)

        # output results to track
        print(csv_dict['runtime_mins'])
        print('accuracy:'+str(csv_dict['accuracy']))
        print('\n\n\n')
        gc.collect()

    return results_list

In [15]:
def plotYvsYhat(y: np.array, yhats: np.array, num_bins: int=10):
    ''' plot average values of y against bins of yhat.

    Args:
        y (np.array): vector of target variable.
        yhats (np.array): vector of fitted values.
        num_bins (int): number of bins to group the yhats into.
    '''
    # form data
    temp_df = pd.DataFrame(data={'y': y, 'yhat': yhats})
    temp_df['yhat_bin'] = pd.qcut(temp_df.yhat, q=num_bins, labels=np.arange(num_bins))

    # plot
    temp_df.groupby('yhat_bin')['y'].mean().plot()

In [16]:
if __name__ == "__main__":
    # set args
    in_fp = '../1-data/clean/bars_btceth_12hour.pkl'
    cv_out_fp = '../3-output/cv_results'

    # read in and prep training+validation data
    df = pd.read_pickle(in_fp)
    df = df.set_index('date')
    df = df.astype('float32')
    df = df[:'2022-03-31']
    
    # run the cv
    cv_results = runCV(df, 
        lhs_col='y_btc_eth_diff_r_tp5_tp730',
        val_start_date='2021-07-01',
        num_cpus=20, 
        arch_name='rf_multiclass', 
        out_csv_fp=cv_out_fp)

    # select best model
    # TODO SELECT THE BEST MODEL

    # report y's vs yhat's distribution
    # TODO plot avg return by the three bins

{'num_estimator': 250, 'subsample': 0.95, 'min_samp_leaf': 50, 'max_depth': 2, 'max_feature': 0.2, 'loss_func': 'log_loss', 'weight_func': 'linear', 'class_balance': 'balanced', 'ret_threshold': 0.001} 



 40%|████      | 220/548 [01:34<02:23,  2.29it/s]

##### Backtest:
    - fit GBDT predicting long or short
    - execute an expanding window CV across all GBDT hyperparameters
        - initialize training data as 2016 thru Q2 2021 and validation data is Q3 2021 through Q1 2022
        - fit on training data and predict on first observation of validation data
        - reset training data to include the next row from the validation data and re-fit+predict.
        - generate yhats for the entire validation period.
    - fit lasso in sample way across 10 different lambdas, take lasso that performs best, fit linear reg on those feats as a benchmark for GBDT to out perform; use lin reg of gbdt dont outperform dis.
    - portfolio optimization: 
        - ex post take all the yhats to form a trading rule mapping from this vector of yhats to vector of 5,4,3,2,1,0,-1,-2,-3,-4,-5 for 5x long to 5x short. 
        - so basically where do i draw these 10 lines in my dist of 0 to 1? 
        - max returns or max sharpe ratio by picking these 10 points, monotoically, between 0 and 1 to create vector of positions. 
        - adjust all of this to take out transaction costs both open+close costs as well as margin costs. maybe these two costs are just vectors as well.
    - calc avg return at 6 hour freq, total return over val period, geom avg return annualized, and sd of simple returns annualized.
    - select highest geom avg return and also one with highest sharpe
    - confirm both have geom avg return statistically different from zero.
        - bootstrap some portfolio return s.e. to calc t stat for backtest return to see if stat sig diff from zero. 
        - given my benchmark is zero. 
        - ensure my validation period is big enough that i could get sig result, i.e. small enough S.E. just randomly generate 10k vectors of positions to calc geom average return; form empirical dist; take standard deviation of this dist to get the s.e. 
        - adjust all of this to take out transaction costs
    - for selected model, go fit in true out of sample period of Q2-Q4 2022 to confirm again geom avg return is stat different from zero.

All of the below code is old code that I will delete once I have completed this backtest file.

In [ ]:
# TODO SOME BOOTSTRAP CODE TO USE LATER

# generate 100 vectors of length 1096 selecting from [-5,-4,-3,-2,-1,0,1,2,3,4,5]
bs_size = 100
periods = 1096
positions = [-5,-4,-3,-2,-1,-0.75,-0.5,-0.25,0,0.25,0.5,0.75,1,2,3,4,5]
portfolios_list = []
for i in range(bs_size):
    np.random.seed(i)
    portfolios_list.append(np.random.choice(positions, size=periods))

# calc returns
btc_eth_diff_array = df[(df.date >= '2021-07-01') & (df.date < '2022-04-01')].y_btc_eth_diff_r_tp5_tp370.values
returns_list = []
for i in range(bs_size):
    portfolio = portfolios_list[i]
    geom_return = np.prod(1+portfolio*btc_eth_diff_array)**(1/periods)-1
    returns_list.append(geom_return)
# calc s.e. of dist
# think thru how to confirm this is enough data

In [1]:
# IMPORT PACKAGES
import pandas as pd
import numpy as np
import itertools
import time
import pickle
from dateutil.relativedelta import relativedelta
from datetime import datetime
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from tqdm import tqdm


In [ ]:
def fitOLS(train_df, lhs_col, rhs_cols):
    # Obtain the training input and output data and, if passed, validation data
    train_rhs   = train_df[rhs_cols].values.astype('float32')
    train_lhs   = train_df[lhs_col].values.reshape(-1).astype('int')

    # add constant
    train_rhs = np.concatenate((np.ones([len(train_lhs),1]), train_rhs),
                               axis=1)

    # fit
    betahat = np.matmul(np.linalg.inv(np.matmul(train_rhs.T, train_rhs)),
                        np.matmul(train_rhs.T, train_lhs))

    return betahat


def genOLSYhats(oos_df, model, lhs_col, rhs_cols):    
    # Obtain the RHS data
    oos_rhs   = oos_df[rhs_cols].values.astype('float32')
    oos_lhs   = oos_df[lhs_col].values.reshape(-1).astype('int')
    
    # add constant
    oos_rhs   = np.concatenate((np.ones([len(oos_lhs),1]), oos_rhs),
                                axis=1).reshape(-1)
    
    # Form the yhats
    yhats = np.dot(model, oos_rhs)
    
    # Return results
    return pd.DataFrame(data={'date': oos_df.index.values,
                              'y': np.array(oos_df['y'].values),
                              'yhats': yhats})

In [ ]:
def fitLasso(train_df, hps_yhats_dict, lhs_col, rhs_cols):
    # Extract hps
    alpha = hps_yhats_dict['alpha']

    # Obtain the training input and output data and, if passed, validation data
    train_rhs   = train_df[rhs_cols].values.astype('float32')
    train_lhs   = train_df[lhs_col].values.reshape(-1).astype('int')

    # standardize
    scaler = StandardScaler()
    train_rhs = scaler.fit_transform(train_rhs)

    # fit
    model = linear_model.Lasso(alpha=alpha, 
                               selection='cyclic')
    model.fit(train_rhs, train_lhs)

    # gather what coefs were used
    used_coefs = (model.coef_!=0)*1

    return model, used_coefs, scaler

def genLassoYhats(oos_df, model, lhs_col, rhs_cols, scaler):    
    # Obtain the RHS data
    oos_rhs   = oos_df[rhs_cols].values.astype('float32')
    oos_lhs   = oos_df[lhs_col].values.reshape(-1).astype('int')
    
    # Standarize
    oos_rhs   = scaler.transform(oos_rhs)
    
    # Form the yhats
    yhats = model.predict(oos_rhs)
    
    # Return results
    return pd.DataFrame(data={'date': oos_df.index.values,
                              'y': np.array(oos_df['y'].values),
                              'yhats': yhats})

In [ ]:
def labelPortfolioWeights(df):
    # set parameters for tertile weights
    # note: can't go to zero on any as there may be week where that is only yhat
    bottom_tertile = 1/6
    mid_tertile = 1/3
    top_tertile = (1-bottom_tertile-mid_tertile)

    # create portfolio weights
    df = df.sort_values(by=['date', 'yhat'])
    df['counts']  = 1
    df['total_assets_per_week_tertile'] = df.groupby(['date', 'yhat']).counts.transform('sum')
    df.loc[df.yhat == -1, 'prtfl_wght'] = bottom_tertile / df[df.yhat == -1].total_assets_per_week_tertile
    df.loc[df.yhat == 0,  'prtfl_wght'] = mid_tertile / df[df.yhat == 0].total_assets_per_week_tertile
    df.loc[df.yhat == 1,  'prtfl_wght'] = top_tertile / df[df.yhat == 1].total_assets_per_week_tertile

    # clean up
    df = df.drop(['counts', 'total_assets_per_week_tertile'], axis=1)

    # fix weeks where portfolio weights add up to less than 1
    temp_df = df.groupby(['date'])[['prtfl_wght']].sum()
    temp_df = temp_df[~np.isclose(temp_df.prtfl_wght, 1)]
    if 0<temp_df.shape[0]:
        for i in range(temp_df.shape[0]):
            date = temp_df.index.values[i]
            total_weight = temp_df.prtfl_wght.values[i]
            df.loc[df.index == date, 'prtfl_wght'] = df[df.index==date].prtfl_wght * (1/total_weight)
            
    # confirm portfolio weights roughly sum to 1 for each week
    assert(len(np.unique(df.index)) == 
           np.sum(np.isclose(df.groupby(['date']).prtfl_wght.sum(), 1,
                             rtol=1e-2, atol=1e-2)))

    return df


In [ ]:
def calcAnnualTransactionCosts(df):
    # merge on the previous week's holdings to the new holdings
    temp_df = df.copy()
    temp_df = temp_df[temp_df.index<np.max(temp_df.index)]
    temp_df.index = temp_df.index+np.timedelta64(1, 'D')
    temp_df = temp_df[['asset', 'prtfl_wght']]
    temp_df = temp_df.rename(columns={'prtfl_wght': 'prtfl_wght_tm7'})
    df      = df.merge(temp_df,
                       on=['date', 'asset'],
                       how='outer',
                       validate='one_to_one')

    # calc weekly turnover and ensure it has the appropriate range
    df['asset_to'] = np.abs(df.prtfl_wght - df.prtfl_wght_tm7)
    to_df = df.groupby('date')[['asset_to']].sum().reset_index()
    assert((np.min(to_df.asset_to)>=0) & (np.max(to_df.asset_to<=2)))

    # correct the first and last week valid for buying the initial port and liquidating
    to_df.loc[0, 'asset_to'] = 1
    to_df = pd.concat((to_df, pd.DataFrame(data={'date': np.max(temp_df.index.values)+np.timedelta64(1, 'D'),
                                                 'asset_to': 1}, index=[0])))
    to_df = to_df.reset_index(drop=True)

    # add transaction costs assuming maker and taker fee of 20 bps each
    to_df['tc'] = to_df.asset_to*0.002

    # return annualize transaction cost
    return -np.sum(to_df.tc)/(to_df.shape[0]/52)

In [ ]:
def max_draw_down(r_df):
    cumulative_ret=(r_df.r+1).cumprod()
    roll_max=cumulative_ret.rolling(len(cumulative_ret), min_periods=1).max()
    daily_drawdown=cumulative_ret/roll_max
    max_daily_drawdown=daily_drawdown.min() - 1
    return max_daily_drawdown

In [ ]:
def max_1_week_loss(r_df):
    max_loss=(r_df['r']+1).rolling(7).apply(np.prod)
    max_loss_minus=max_loss.min()-1
    return max_loss_minus

In [ ]:
def genTestYhats(df, opt_hps, test_year=2022): 
    test_dates = np.unique(df[df.index.year == test_year].index.values)

    all_cols = list(df.columns.values)
    lhs_col  = ['y']
    all_cols.remove('y')
    rhs_cols = all_cols

    def loopOverOOSDates(test_date):
        # build data
        temp_df       = df[df.index <= test_date].copy()
        train_df      = temp_df[temp_df.index < test_date]
        oos_df        = temp_df[temp_df.index == test_date]

        # fit and predict
        model          = fitGBDT(train_df, opt_hps, lhs_col, rhs_cols)
        oos_results_df = genYhats(oos_df, model, lhs_col, rhs_cols)

        return oos_results_df

    # run
    oos_results = Parallel(n_jobs=num_cpus)(delayed(loopOverOOSDates)(test_date) for test_date in tqdm(test_dates))

    # Extract oos results
    oos_results_df = pd.concat(oos_results)

    return oos_results_df

In [ ]:
# RUN THE CODE

# Load in the data
input_fp = '../3-data/clean/panel_btceth_30min.pkl'
df = pd.read_pickle(input_fp)

# Clean up the data 
df = df.set_index('date')
df = df.astype('float32')

# Select useful variables accoridng to Lasso
df = df[['y',
         'covar_btc_volume_sum_tm288',
         'covar_btc_r_tm36',
         'covar_btc_ema_144_t',
         'covar_btc_ema_288_t',
         'covar_eth_r_tm12',
         'covar_eth_ma_24_t',
         'covar_btc_rsi_tm288',
         'covar_eth_rsi_tm2016',
         'covar_eth_rsi_tm4032',
         'macro_mcap_ret_ath_t',
         'macro_mcap_altcoin_ret_ath_t']]

In [ ]:
# Run CV
results_list = runCV(df, last_train_year=2019, val_end_year=2021,
                     arch_name='lasso_longshort', num_cpus=20)


In [ ]:
if __name__ == "__main__":
    # step 1

    # step 2

    # step 3

    # do gbdt with the full set where i randomly sample sqrt of them and then try with the 36 cols from lasso where i sample half or something
    
    # TODO explore if normalization helps or not and make sure no forward looking bias
# played with the lasso code below to select a final set of 36 features
    lasso_feats =  ['covar_btc_covar_trades_t_ma_tm8640',
                    'covar_btc_covar_trades_t_min_tm30',
                    'covar_btc_covar_trades_t_min_tm60',
                    'covar_btc_covar_volume_t_min_tm4320',
                    'covar_btc_p_log_t',
                    'covar_btc_r_1min_ema_tm360',
                    'covar_btc_r_5min_skew_tm720',
                    'covar_btc_r_tm1440',
                    'covar_btc_r_tm15',
                    'covar_btc_r_tm86400',
                    'covar_btc_rsi_tm720',
                    'covar_eth_covar_trades_t_max_tm4320',
                    'covar_eth_covar_trades_t_min_tm20',
                    'covar_eth_covar_trades_t_vol_tm60',
                    'covar_eth_covar_volume_t_ma_tm60',
                    'covar_eth_covar_volume_t_ma_tm8640',
                    'covar_eth_covar_volume_t_max_tm4320',
                    'covar_eth_covar_volume_t_max_tm720',
                    'covar_eth_covar_volume_t_sum_tm20',
                    'covar_eth_covar_volume_t_sum_tm60',
                    'covar_eth_covar_volume_t_vol_tm360',
                    'covar_eth_p_t',
                    'covar_eth_r_1min_ema_tm60',
                    'covar_eth_r_1min_ma_tm360',
                    'covar_eth_r_1min_vol_tm20',
                    'covar_eth_r_1min_vol_tm5',
                    'covar_eth_r_5min_ma_tm20160',
                    'covar_eth_r_5min_ma_tm60',
                    'covar_eth_r_5min_min_tm30',
                    'covar_eth_r_5min_min_tm720',
                    'covar_eth_r_cummax_t',
                    'covar_eth_r_cummin_t',
                    'covar_eth_r_tm120',
                    'covar_eth_r_tm1440',
                    'covar_eth_r_tm40320',
                    'covar_eth_volume_t']

In [ ]:
results_list_master = results_list.copy()

In [ ]:
# # SAVE RESULTS
# with open('../3-data/derived/validation_2021_results_btceth_ols.pickle', 'wb') as handle:
#     pickle.dump(results_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# study for signal
for i in [6, 5, 4, 3, 2, 1, 0]:
    print(i)
    df             = pd.read_pickle(input_fp)
    val_results_df = results_list[i]['results_df']
    val_results_df = val_results_df.drop('y', axis=1)
    val_results_df = val_results_df.merge(df,
                                          on=['date'],
                                          how='inner', 
                                          validate='one_to_one')
    val_df = pd.DataFrame(data={'date': val_results_df.date.values,
                                'y': val_results_df.y.values,
                                'y_btc_eth_r_tp2_tp7': val_results_df['y_btc_eth_r_tp2_tp7'].values,
                                'yhat': val_results_df.yhats.values})

    val_df['yhat'] = (val_df.yhat-np.min(val_df.yhat))
    val_df['yhat'] = val_df.yhat/np.max(val_df.yhat)
    for lev in [2, 3, 4, 5]:
        for fee in [1e-4, 2e-4, 5e-4]:
            print(lev)
            print(fee)
            yhat_vals = val_df.yhat.values
            for yhat_val in yhat_vals:
                val_df.loc[val_df.yhat<=yhat_val, 'pw'] = -1
                val_df.loc[val_df.yhat>yhat_val, 'pw'] = 1
                assert(0==val_df.pw.isnull().sum())
                port_ret = np.prod(lev*(1+val_df.pw*val_df.y_btc_eth_r_tp2_tp7)-fee)**(1/len(val_df))-1
                val_df.loc[val_df.yhat==yhat_val, 'ret'] = port_ret

            val_df = val_df.drop('pw', axis=1)

            val_df = val_df.sort_values(by='yhat')
            val_df['yhat'] = np.arange(len(val_df))/len(val_df)
            plt.plot(val_df.yhat, val_df.ret)
            plt.show()

In [ ]:
bins = 10
val_df['yhat_decile'] = pd.qcut(val_df.yhat, q=bins, labels=np.arange(bins))
val_df.groupby('yhat_decile')['y_btc_eth_r_tp2_tp7'].mean().plot()

In [ ]:
# study feature importance

feat_imprt = np.zeros(df.shape[1]-2)
models = results_list[0]['models']
for model in models:
    feat_imprt += model.feature_importances_
feat_imprt = feat_imprt/(len(models)-1)
print(feat_imprt)

# useful features:
print('useful features:')
threshold = np.median(feat_imprt)
for i in np.where(feat_imprt > threshold)[0]:
    print(df.columns[2:][i])
print('\nuseless features:')
for i in np.where(feat_imprt <= threshold)[0]:
    print(df.columns[2:][i]) 

In [ ]:
# form validation period results
val_results_df = results_list[0]['results_df']
val_results_df = val_results_df.drop('y', axis=1)
val_results_df = val_results_df.merge(df,
                                      on=['date'],
                                      how='inner', 
                                      validate='one_to_one')
val_df = pd.DataFrame(data={'date': val_results_df.date.values,
                            'y': val_results_df.y.values,
                            'y_btc_eth_r_tp2_tp7': val_results_df['y_btc_eth_r_tp2_tp7'].values,
                            'yhat': val_results_df.yhats.values})

In [ ]:
# Very nice!
val_df['yhat_decile'] = pd.qcut(val_df.yhat, q=10, labels=np.arange(10))
val_df.groupby('yhat_decile')['y_btc_eth_r_tp2_tp7'].mean().plot()

In [ ]:
# Explore different trading rules
trading_cost = 0.001

max_return = 0

# loop over leverage
for leverage in [2, 3, 4, 5, 6, 7, 8, 9, 10]:
    for long_threshold in range(10):
        for short_threshold in range(10):
            # grab yhats
            yhats = val_df.yhat.values

            # figure out thresholds
            neg_yhats = yhats[yhats<0]
            pos_yhats = yhats[yhats>=0]
            neg_threshold = pd.qcut(neg_yhats, q=10).categories[short_threshold].right
            pos_threshold = pd.qcut(pos_yhats, q=10).categories[long_threshold].left

            # turn yhats into actual long or short position
            pos = np.piecewise(yhats, [yhats<=neg_threshold, 
                                       (yhats>neg_threshold)&(yhats<=pos_threshold), 
                                       yhats>pos_threshold],
                               [-1, 0, 1])

            # figure out places to charge me
            # by finding indices where previous value is different and the index value is not 0
            trading_fee_array = np.concatenate((np.array([trading_cost]), 
                                               ((pos[1:]!=pos[:-1])&(pos[1:]!=0))*1*trading_cost))

            # calc returns after cost
            returns = pos*val_df.y_btc_eth_r_tp2_tp7*leverage
            returns = returns - trading_fee_array

            # report
            overall_return = np.prod(returns+1)-1
            if overall_return > max_return:
                max_return = overall_return
                
                print('leverage '+str(leverage))
                print('long thres '+str(long_threshold))
                print('short threshold '+str(short_threshold))
                
                print('overal return '+str(np.round(overall_return, 4)))
                start_index = 0
                for i in range(int(2*24*30.5), 10272, int(2*24*30.5)):
                    end_index = i
                    month_returns = returns[start_index:end_index]
                    month_returns = np.prod(month_returns+1)-1

                    start_index = i
                    print('months return ' +str(np.round(month_returns, 4)))

                # report return adjusted by std of negative returns
                std_neg_returns = np.std(returns[returns<0])
                print('adjusted sharpe '+str(np.round(overall_return/std_neg_returns,4)))

                # max dd
                cumulative_ret=(returns+1).cumprod()
                roll_max=cumulative_ret.rolling(len(cumulative_ret), min_periods=1).max()
                dd=cumulative_ret/roll_max
                max_dd=dd.min() - 1
                print('max dd '+str(np.round(max_dd, 4)))

                # sharpe
                sharpe = np.mean(returns)/np.std(returns)*np.sqrt(365.25*24*2)
                print('sharpe '+str(np.round(sharpe, 4)))
                print('\n\n')

In [ ]:

possible_trades = [5, 4, 3, 2, 1, 0.75, 0.5, 0.25, -0.25, -0.5, -0.75, -1, -2, -3, -4, -5]

# initialize objects
max_sharpe = 0
optimal_cutoffs = list(np.zeros(len(possible_trades)))
optimal_positions = np.zeros(len(y))

# determine optimal cutoff for max long
trade = possible_trades[0]
possible_cutoffs = np.flip(np.sort(yhats))
optimal_cutoffs[0] = np.max(possible_cutoffs)

for cutoff in possible_cutoffs:
    positions = np.where(yhats >= cutoff, trade, optimal_positions)
    tcs = calcTransactionCosts(positions)
    returns = positions*y-tcs
    sharpe  = calcSharpe(returns, periods_in_year=365*4)
    if sharpe > max_sharpe:
        max_sharpe = sharpe
        print(calcGeomAvg(returns, annualized=True, periods_in_year=365*4))
        optimal_cutoffs[0] = cutoff

# update optimal positions
optimal_positions = np.where(yhats >= optimal_cutoffs[0], trade, optimal_positions)

# determine optimal cutoffs for long positions
for i in range(1,8):
    trade = possible_trades[i]
    prev_cutoff = optimal_cutoffs[i-1]
    possible_cutoffs = np.flip(np.sort(yhats[yhats <= prev_cutoff]))
    optimal_cutoffs[i] = np.max(possible_cutoffs)
    for cutoff in possible_cutoffs:
        positions = np.where((yhats < prev_cutoff) & (yhats >= cutoff), trade, optimal_positions)
        tcs = calcTransactionCosts(positions)
        returns = positions*y-tcs
        sharpe  = calcSharpe(returns, periods_in_year=365*4)
        if sharpe > max_sharpe:
            max_sharpe = sharpe
            print(calcGeomAvg(returns, annualized=True, periods_in_year=365*4))
            optimal_cutoffs[i] = cutoff

    # update optimal positions
    optimal_positions = np.where((yhats < prev_cutoff) & (yhats >= optimal_cutoffs[i]), trade, optimal_positions)

In [ ]:
# TODO THERE IS MORE TO GRAB FROM v-benchmark_gbdt_btceth_ls.ipynb as needed

In [ ]:
# for year in [2016, 2017, 2018, 2019, 2020, 2021]:
#     lasso_df = df[df.date.dt.year == year].copy()

#     # set lasso parameters
#     rhs_feats = list(included_feats)
#     lhs_col   = 'y_btc_eth_diff_r_tp5_tp370'
#     alpha     = 0.001

#     # scale RHS
#     y = lasso_df[lhs_col].values
#     X = lasso_df[rhs_feats].values
#     X_scaled = StandardScaler().fit_transform(X)

#     # fit
#     model = Lasso(alpha=alpha, fit_intercept=False)
#     model.fit(X_scaled, y)

#     # show selected feats
#     lasso_feats = [rhs_feats[i] for i in np.nonzero(model.coef_)[0]]
#     print(year)
#     print(lasso_feats)

In [ ]:

# TODO MOVE ANY FUNCTIONS THAT I MAY USE ELSEWHERE TO COMMON FOLDER SHARED ACROSS PROJECTS THAT I JUST IMPORT WHEN NEEDED